In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
import folium
import warnings
import pickle
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# load data for network creation
# read data from file
datasize = 'full'
location = 'oslo'
data_date = '202208'
eval_date = '202209'

orig_filename = '../data/processed/'+data_date+'_points_'+location+'_cleaned_meta_'+datasize+'_dualSplit_2.parquet'
gdf = gpd.read_parquet(orig_filename)
# Transform to desired CRS
# 4326 for WGS 84 (global) // 32632 for UTM 32N (Norway)
crs = 32632  # Coordinate reference system
gdf.to_crs(crs, inplace=True)  # Transformation

In [3]:
# load evaluation data
eval_file = eval_date+'_points_'+location+'_cleaned_meta_full_dualSplit_2'
filename = '../data/processed/' + eval_file + '.parquet'
test_gdf = gpd.read_parquet(filename)
crs = 32632  # Coordinate reference system
test_gdf.to_crs(crs, inplace=True)  # Transformation
all_test_trajectories = mpd.TrajectoryCollection(test_gdf, traj_id_col='mmsi', obj_id_col='mmsi')

# select evaluation data
selection_start = 0
selection_end = len(all_test_trajectories)
selection_step = 10
selection = np.arange(selection_start, selection_end, selection_step)
n_trajectories = len(selection)
mmsis = test_gdf.mmsi.unique()[selection]
test_trajectories = all_test_trajectories.filter('mmsi', mmsis.tolist?

In [11]:
waypoints = [1696, 1766, 1697, 1655, 1841, 1865, 1872, 118, 114, 1862, 1865, 1871, 1872, 1868, 1869, 
 2034, 2035, 2032, 2022, 2023, 2153, 2151, 2152]

In [12]:
visited = set()
new_path = []

for waypoint in waypoints:
    if waypoint not in visited:
        visited.add(waypoint)
        new_path.append(waypoint)
    else:
        # Remove waypoints after the loop point
        new_path = new_path[:new_path.index(waypoint)+1]

print(new_path)

ValueError: 1872 is not in list